- The ipyparallel controller creates two files which contain information how to connect to the controller (address and authentication secrets).
- To to connect our engines and clients to the cluster, we need to copy those files to our local machine.
- We use `rsync`.

The address changes, so replace `1.2.3.4` by the address shown on the projector.

In [2]:
!rsync -v 'student@172.25.224.50:.ipython/profile_default/security/ipcontroller-*.json' ~/.ipython/profile_default/security/

The authenticity of host '172.25.224.50 (172.25.224.50)' can't be established.
ED25519 key fingerprint is SHA256:UPJArdVFh2z+vqTUJNA9s4/SqHydNQe59DQnT0wX0zc.
This key is not known by any other names
Are you sure you want to continue connecting (yes/no/[fingerprint])? ^C


In [ ]:
.Those files are just JSON. We can print them with syntax highlighting with `pygmentize`:

In [ ]:
!pygmentize /home/student/.ipython/profile_default/security/ipcontroller-engine.json 

In [ ]:
!pygmentize /home/student/.ipython/profile_default/security/ipcontroller-client.json 

This starts `htop` in a terminal window to monitor running processes.

If you prefer, start `htop` manually in some terminal window.

In [3]:
!bash -c 'urxvt -e htop & sleep 3'

urxvt: unable to calculate font width for ':slant=0:weight=200:pixelsize=13:minspace=True:lang=he', using max_advance_width.


Now we start our ipcluster engines on the local machine.
They will connect to the controller configured by the JSON `ipcontroller-engine.json`. 

In [4]:
!bash -c 'urxvt -e ipcluster engines & sleep 3'

In [5]:
import ipyparallel as ipp
rc = ipp.Client()

In [6]:
rc.ids

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [7]:
def hostname():
    import socket
    return socket.gethostname()

In [8]:
rc[:].apply_sync(hostname)

['krowka',
 'krowka',
 'krowka',
 'krowka',
 'aspp07',
 'aspp07',
 'aspp07',
 'aspp07',
 'aspp02',
 'aspp02',
 'aspp02',
 'aspp02',
 'aspp15',
 'aspp15',
 'aspp15',
 'aspp15',
 'aspp16',
 'aspp16',
 'aspp16',
 'aspp16']

In [ ]:
def f(x):
    return x**4 - 3*x

In [18]:
import numpy as np
from multiprocessing.pool import Pool as ProcessPool

def crunch_numbers(x):
    y = 0.0
    for i in range(int(x*100)):
        y += (i**7)/(i+1)
    return y

input_data = np.arange(1,5)


In [20]:
%%time
results = rc[:].map_sync(crunch_numbers,input_data)
results

CPU times: user 103 ms, sys: 6.23 ms, total: 109 ms
Wall time: 24.7 s


[13630931940874.816,
 1786286903191754.5,
 3.0759763719752636e+16,
 2.313387343876235e+17]

In [24]:
def usage():
    import subprocess
    return subprocess.check_output('free', text=True)
    
usage()

'               total        used        free      shared  buff/cache   available\nMem:         7929372     4430604     1873728      431704     1625040     2738256\nSwap:        7928828       93184     7835644\n'

In [ ]:
def integrate(a, b, n):
    dx = (b - a) / n
    s = 0.0
    x = a + 0.5 * dx
    for i in range(n):
        y = f(x)
        s += y
        x += dx
    return s * dx

In [ ]:
rc[:].push(dict(f=f)).wait()

In [ ]:
stepcounts = list(range(10_000, 20_000, 10))
a = [-2 for _ in stepcounts]
b = [+2 for _ in stepcounts]

In [ ]:
%timeit -r 1 -n 1    list(map(integrate, a, b, stepcounts))

In [ ]:
%timeit -r 1 -n 1    rc[:].map_sync(integrate, a, b, stepcounts)